Views Notebook (Part 2)

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

import pyspark 
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a local Spark cluster with two executors (if it doesn't already exist)
spark_session = SparkSession.builder.master('local[2]').getOrCreate()
sc = spark_session.sparkContext


In [ ]:
from pyspark.sql.functions import col, window, to_json, struct
from pyspark.sql.types import TimestampType

# Load cleaned stream from Kafka 
cleaned_stream = (
    spark_session.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "ingest-cleaned")
    .option("startingOffsets", "latest")
    .load()
    .selectExpr("CAST(value AS STRING) AS json")
    .selectExpr("from_json(json, 'timestamp TIMESTAMP') as data")
    .select("data.*")
)

Events per day

In [ ]:
daily_counts = (
    cleaned_stream
    .withWatermark("timestamp", "30 minutes")
    .groupBy(window(col("timestamp"), "1 day"))
    .count()
)

tq = (
    daily_counts.writeStream
    .outputMode("append")
    .format("memory")
    .queryName("events_per_day")
    .start()
)

To show/display

In [ ]:

spark_session.sql("SELECT * FROM events_per_day").show()

Part 2 Events per hour with delta <-> previous day  

In [ ]:
#creating the file for the static dataframe
# Define the file path
file_path = "data/historical_hourly.csv"

# Check if the file exists
if not os.path.exists(file_path):
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
     # Write an empty CSV file with only the header
    with open(file_path, "w") as file:
        file.write("hour,historical_count\n")
    print(f"Empty file created at {file_path}")
else:
    print(f"File already exists at {file_path}")
    

In [ ]:
# Load static historical data
historical_df = (
    spark_session.read.format("csv")
    .option("header", "true")
    .load("data/historical_hourly.csv")
    .withColumn("hour", col("hour").cast(TimestampType()))
    .withColumn("historical_count", col("historical_count").cast("long"))
)

# Compute current hourly counts
hourly_counts = (
    cleaned_stream
    .withWatermark("timestamp", "30 minutes")
    .groupBy(window(col("timestamp"), "1 hour"))
    .count()
    .withColumnRenamed("count", "current_count")
    .withColumn("hour", col("window.start"))
)

# Join with historical data and compute delta
hourly_with_delta = (
    hourly_counts
    .join(historical_df, on="hour", how="left")
    .withColumn("delta", col("current_count") - col("historical_count"))
    .select("hour", "current_count", "historical_count", "delta")
)

# Write to Kafka topic "hourly-delta"
output_to_kafka = (
    hourly_with_delta
    .select(to_json(struct("hour", "current_count", "historical_count", "delta")).alias("value"))
    .writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("topic", "hourly-delta")
    .option("checkpointLocation", "checkpoints-hourly-delta")
    .outputMode("append")
    .start()
)


Wait for stream to finish

In [ ]:

output_to_kafka.awaitTermination()